# PRED-SQL MANUAL SELECTION EXPERIMENT

In [ ]:
import os
import json
import nest_asyncio
from tqdm import tqdm
from utils.prompts.prompt_builder import prompt_factory
from utils.data.data_builder import load_data
from utils.llm.ask_llm import run_llm
from third_party.spider_eval.evaluation import evaluate_spider
from utils.data.post_process import save_results
from utils.prompts.prompt_builder import get_openai_key

EMBEDDING = None

In [1]:
from openai import OpenAI

# Set the environment variable to provide access to the OpenAI API (set in utils/parameters.py)
os.environ['OPENAI_API_KEY'] = get_openai_key()

# Create the OpenAI client
client = OpenAI()

In [3]:
path_data = "benchmarks"

data = load_data("spider", path_data, pre_test_result= 'data/pre-predictions/dail-sql+GPT-4.txt')

## Pred-SQL Manual Selection Spider 1-Shot Experiment

In [ ]:
prompt = prompt_factory(k_shot = 1, repr_type= "SQL", example_format= "QA", selector_type= "MANUALPRED")(data=data, tokenizer="gpt-3.5-turbo")

In [ ]:
# Format all Spider test questions for 1-Shot GPT-3.5-Turbo Text-to-SQL conversion
questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    questions.append(question_format)

In [16]:
# Define the task parameters for saving the questions for the Spider baseline experiment

args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 1,
    "example_type": "QA",
    "selector_type": "MANUALPREDSQL",
    "embedding_model": None
}

# Define the task object for saving the questions for the Spider baseline experiment

task = {
        "args": args,
        "questions": questions
    }

OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "1-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-1.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [4]:
OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "1-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-1.json")

RESPONSES_FILE = os.path.join(OUT_DIR, "pred-sql-manual-responses-1.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [23:01<00:00,  1.34s/it]  


In [5]:
nest_asyncio.apply()

OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "1-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-1.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "pred-sql-manual-responses-1.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"pred-sql-manual-results-1.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.907                0.841                0.736                0.524                0.788               


## Pred-SQL Manual Selector Spider 3-shot Experiment

In [ ]:
prompt = prompt_factory(k_shot = 3, repr_type= "SQL", example_format= "QA", selector_type= "MANUALPRED", embedding_model= None)(data=data, tokenizer="gpt-3.5-turbo")

In [ ]:
# Format all Spider test questions for 3-Shot GPT-3.5-Turbo Text-to-SQL conversion
questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    questions.append(question_format)

In [ ]:
# Define the task parameters for saving the questions for the Spider baseline experiment

args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 3,
    "example_type": "QA",
    "selector_type": "MANUALPREDSQL",
    "embedding_model": None
}

# Define the task object for saving the questions for the Spider baseline experiment

task = {
        "args": args,
        "questions": questions
    }

OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "3-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-3.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [6]:
OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "3-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-3.json")

RESPONSES_FILE = os.path.join(OUT_DIR, "pred-sql-manual-responses-3.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [15:11<00:00,  1.13it/s]


In [7]:
nest_asyncio.apply()

OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "3-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-3.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "pred-sql-manual-responses-3.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"pred-sql-manual-results-3.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.907                0.843                0.713                0.512                0.783               


## Pred-SQL Manual Experiment Spider 5-Shot Experiment

In [ ]:
prompt = prompt_factory(k_shot = 5, repr_type= "SQL", example_format= "QA", selector_type= "MANUALPRED", embedding_model= 'None')(data=data, tokenizer="gpt-3.5-turbo")

In [ ]:
# Format all Spider test questions for 5-Shot GPT-3.5-Turbo Text-to-SQL conversion
questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    questions.append(question_format)

In [ ]:
# Define the task parameters for saving the questions for the Spider baseline experiment

args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 5,
    "example_type": "QA",
    "selector_type": "MANUALPREDSQL",
    "embedding_model": None
}

# Define the task object for saving the questions for the Spider baseline experiment

task = {
        "args": args,
        "questions": questions
    }

OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "5-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-5.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [8]:
OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "5-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-5.json")

RESPONSES_FILE = os.path.join(OUT_DIR, "pred-sql-manual-responses-5.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [20:35<00:00,  1.19s/it] 


In [9]:
nest_asyncio.apply()

OUT_DIR = os.path.join("chapter-4","results", "pred-sql-manual-experiments", "5-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "pred-sql-manual-prompts-5.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "pred-sql-manual-responses-5.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"pred-sql-manual-results-5.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.907                0.841                0.747                0.518                0.789               
